# Visualizando Predicciones de Cuenca

Se importan las librerías

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import LSTM

Se importan los datos

In [12]:
df = pd.read_csv('./final_train_2015.csv', index_col=0, parse_dates=True)
df = df.asfreq('h')
df = df.between_time('08:00:00','17:00:00')
df = df[['SRGLOAVG1H','TAMAX1H','TAMIN1H','WS1HX','WS1HA','RHMIN1H','RHMAX1H']]

In [14]:
df_test = pd.read_csv('./final_test_2015.csv', index_col=0, parse_dates=True)
df_test = df_test.asfreq('h')
df_test = df_test[['SRGLOAVG1H','TAMAX1H','TAMIN1H','WS1HX','WS1HA','RHMIN1H','RHMAX1H']]
df_test = df_test.between_time('08:00:00','17:00:00')

In [15]:
from sklearn.preprocessing import MinMaxScaler

In [16]:
scaler = MinMaxScaler()
scaler.fit(df)
scaled_train = scaler.transform(df)
scaled_test = scaler.transform(df_test)

Se importan los mejores modelos

In [5]:
best_univariado = keras.models.load_model('../../../Best_models/cuenca/LSTM_univariado_RS.h5')

In [6]:
best_multivariado = keras.models.load_model('../../../Best_models/cuenca/LSTM_multivariado_RS.h5')

Se establecen los coeficientes del modelo de referencia AR.

In [9]:
coef = [88.465247,
                0.635393,-0.103026,0.043116,-0.032294,
                -0.004597,-0.002247,-0.010747,0.021191,
                0.111859,0.140868,0.000666,-0.033811,
                -0.006970,-0.041583,-0.016905,-0.012530,
                0.000388,0.007379,0.072742,0.063291,0.011766,
                -0.066640]

Se crea una función para aplicar la ecuación AR.

In [11]:
def get_pred(coef,data):
    prediction = coef[0] # constante
    # los datos se reciben en forma cronológica
    j = 1
    for i in range(len(data)-1,-1,-1):
        prediction += coef[j] * data[i]
        j+=1
    return prediction